In [1]:
#pip install spotipy

In [2]:
#pip install tqdm

In [3]:
# import libraries

print('initializing...')

import configparser
from configparser import ConfigParser
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import time
from tqdm import tqdm

initializing...


In [4]:
# place the config file outside of the current directory
# this will set to the config file as the parent directory

curr_path = os.getcwd()
config_path = os.path.abspath(os.path.join(curr_path, os.pardir))
config = configparser.ConfigParser()
config.read(config_path+'\\'+'spotify_data_proj.ini')


# set up client credentials

client_id = config['credentials']['client_id']
client_secret = config['credentials']['client_secret']
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
# get the number of batches from the user

num_batches = int(input('each batch contains track data for 50 artists. how many batches would you like to retrieve?'))


# get a list of the top artist IDs, 50 at a time

offst=0
top_artist_list = []

for i in tqdm(range(0,num_batches), desc = 'generating list of top artists'):
    top_artists = sp.search(q='year:2023' , type='artist', offset = offst, limit=50)
    for artist in top_artists['artists']['items']:
        top_artist_list.append(artist['id'])
    offst=offst+50
    time.sleep(0.1)

each batch contains track data for 50 artists. how many batches would you like to retrieve? 5


generating list of top artists: 100%|██████████| 5/5 [00:02<00:00,  1.85it/s]


In [6]:
# open a JSON file for writing

with open('spotify_data.json', 'w') as f:
    f.write('[')
    
    # retrieve the track data from the API for each artist
    
    for i in tqdm(range(len(top_artist_list)), desc = "retrieving track data for each artist"):
        artist_tracks = sp.artist_top_tracks(artist_id=top_artist_list[i], country="US")["tracks"]

        # write the data for each track to the JSON file
        
        for track in artist_tracks:                
            json.dump(track, f)
            f.write(',')
            f.write('\n')

        offst = offst+50
        time.sleep(0.1)
        
    f.write(']')

print('data retrieval complete')

retrieving track data for each artist: 100%|██████████| 250/250 [01:03<00:00,  3.93it/s]

data retrieval complete
